In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri setini yükleme
train_data_dir = r"C:\Users\golde\Desktop\data2\Data\train"
validation_data_dir = r"C:\Users\golde\Desktop\data2\Data\valid"
test_data_dir = r"C:\Users\golde\Desktop\data2\Data\test"

# Veri ön işleme işlemleri için ImageDataGenerator kullanımı
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# ResNet-50 modelinin eklenmesi
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))


test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


model.save("resnet_model.h5")
print("Model kaydedildi.")



KeyboardInterrupt



In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Modeli yükle
model = tf.keras.models.load_model('resnet_model.h5')

# Fotoğrafın yolu
photo_path = r"C:\Users\golde\Desktop\archive\xxss.jpg"  # Fotoğrafın yolunu burada belirtin

# Fotoğrafı yükleyip ön işleme yapma
img = image.load_img(photo_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array = img_array / 255.0  # Veriyi [0, 1] aralığına ölçekleme

# Tahmin yapma
predictions = model.predict(img_array)

# Sonucu kontrol etme
if predictions[0][0] >= 0.5:
    prediction = 'sağlıklı'
else:
    prediction = 'kanserli'

print(f"Fotoğrafın sonucu: {prediction}")


1/1 [==============================] - 1s 851ms/step
Fotoğrafın sonucu: kanserli
